In [ ]:
import numpy as np
import pandas as pd
from keras.utils import image_dataset_from_directory

In [ ]:
df_train=pd.read_csv("/kaggle/input/uw-madison-gi-tract-image-segmentation/train.csv")
df="/kaggle/input/uw-madison-gi-tract-image-segmentation/train"

In [ ]:
training_data=image_dataset_from_directory(
df,
seed=123,
shuffle=True,
image_size=(256,256),
subset="training",
validation_split=0.2
)
validation_data=image_dataset_from_directory(
df,
seed=123,
shuffle=True,
image_size=(256,256),

subset="validation",
validation_split=0.2
)

In [ ]:
for image,label in training_data.take(1):
    print(np.unique(label))

# # Deep Learning

In [ ]:
from keras.models import Model
from keras.layers import Dense,Conv2D,MaxPooling2D,TimeDistributed,BatchNormalization,Input,Flatten
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping

In [ ]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [ ]:
num_classes = len(training_data.class_names)
num_classes

In [ ]:
inputs = Input(shape=(256, 256, 3))
C = Conv2D(32,(3,3), activation="relu", kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01), padding="same")(inputs)
C = Conv2D(32,(3,3), activation="relu", kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01), padding="same")(C)
M = MaxPooling2D(pool_size=(2, 2))(C)
B = BatchNormalization()(M)
C = Conv2D(64,(3,3), activation="relu", kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01), padding="same")(B)
C = Conv2D(64,(3,3), activation="relu", kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01), padding="same")(C)
M = MaxPooling2D(pool_size=(2, 2))(C)
F=Flatten()(M)
D=Dense(32, activation="relu", kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01))(F)
D=Dense(32, activation="relu", kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01))(D)

outputs = Dense(85,activation="softmax")(D)
model = Model(inputs, outputs)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(training_data,epochs=50,validation_data=validation_data,callbacks=[early_stopping])

In [ ]:
loss,accuracy=model.evaluate(validation_data)


In [ ]:
cv=cv2.imread("/kaggle/input/uw-madison-gi-tract-image-segmentation/train/case101/case101_day20/scans/slice_0001_266_266_1.50_1.50.png")
cv_resize=cv2.resize(cv,(256,256))
expand_dim=np.expand_dim(cv,axis=-1)
y_pred=model.predict(expand_dim)
y_pred